# Tomographic mapping notebook  
__Written by Haixing Fang, Jon Wright and James Ball__  
__Date: 21/02/2025__

This notebook will try to perform a point-by-point strain refinement from your tomographic-derived grain shapes.  

### NOTE: It is highly recommended to run this notebook on a Jupyter server with many cores and a lot of RAM.  
The compute_origins() function in particular runs locally and can be compute-intensive for large datasets.  
If this is a big scan (e.g 100 million+ 2D peaks), you should definitely refine on the cluster rather than locally.

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

# python environment stuff
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# dataset file to import
dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

# which phase to refine
phase_str = 'Si'

# default options for the single-valued map (shouldn't need to modify this)
default_npks = 20
default_nuniq = 20

# refinement tolerances
hkl_tol_origins = 0.05
hkl_tol_refine = 0.1
hkl_tol_refine_merged = 0.05
ds_tol = 0.004
ifrac = 7e-3
rings_to_refine = None  # can be a list of rings

# use cluster for refinement or run locally?
use_cluster = False

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import ImageD11.sinograms.dataset
from ImageD11.sinograms.sinogram import read_h5
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.sinograms.point_by_point import PBPRefine

import ImageD11.nbGui.nb_utils as utils

%matplotlib ipympl

# Load data
## Dataset

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_path)
print(ds)

## Phases
If the parameter file was a json, we can access the unit cells via `ds.phases.unitcells`

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
ucell = ds.phases.unitcells[phase_str]
print(ucell)

## Peaks

In [ ]:
cf_2d = ds.get_cf_2d()
ds.update_colfile_pars(cf_2d)  # computes geometry, needed for filtration
print(f"Read {cf_2d.nrows} 2D peaks")

## Grains

In [ ]:
grainsinos = read_h5(ds.grainsfile, ds, phase_str)
grains = [gs.grain for gs in grainsinos]

## TensorMap

In [ ]:
tensor_map = TensorMap.from_h5(ds.grainsfile, h5group='TensorMap_' + phase_str)
tensor_map.plot('phase_ids')

In [ ]:
# make a PBPMap from our TensorMap
# fills voxels that have grains with npks = 20 and nuniq = 20
pmap = tensor_map.to_pbpmap(z_layer=0, default_npks=default_npks, default_nuniq=default_nuniq)

In [ ]:
pmap.choose_best(1)
pmap.plot_best(1)

# Refinement

In [ ]:
# set up a refinement manager object
y0 = grainsinos[0].recon_y0
refine = PBPRefine(dset=ds, y0=y0, hkl_tol_origins=hkl_tol_origins, hkl_tol_refine=hkl_tol_refine, hkl_tol_refine_merged=hkl_tol_refine_merged, ds_tol=ds_tol, ifrac=ifrac, phase_name=phase_str, forref=rings_to_refine)
# change the default paths of the refinement manager to append the phase name
refine.own_filename = os.path.splitext(refine.own_filename)[0] + f'_{phase_str}.h5'
refine.icolf_filename = os.path.splitext(refine.icolf_filename)[0] + f'_{phase_str}.h5'
refine.pbpmap_filename = os.path.splitext(refine.pbpmap_filename)[0] + f'_{phase_str}.h5'
refine.refinedmap_filename = os.path.splitext(refine.refinedmap_filename)[0] + f'_{phase_str}.h5'

In [ ]:
# tell it which point-by-point map we are refining
refine.setmap(pmap)

# or load from disk:
# refine.loadmap()

In [ ]:
# set the mask from minimum peak values
# anything greater than 0 should be accepted
refine.mask = pmap.best_npks > 0

In [ ]:
# generate a single-valued map to refine on
refine.setsingle(refine.pbpmap, minpeaks=1)

In [ ]:
# choose 2D peaks to refine with
refine.setpeaks(cf_2d)

# or load from disk:
# refine.loadpeaks()

## Setting up peaks

In [ ]:
# plot the peaks you selected
refine.iplot()

## Compute peak diffraction origins

In [ ]:
# compute diffraction origins - these will be added as a column to refine.icolf
# will then save the new column to disk to avoid re-computation
refine.get_origins()

## Run refinement

In [ ]:
# run the refinement
# if compute_origins took more than a couple of minutes to run, I suggest setting use_cluster=True below
# otherwise if you asked for lots of cores and RAM on this Jupyter instance, you can run it locally (use_cluster=False)
refine.run_refine(use_cluster=use_cluster, pythonpath=PYTHONPATH)

# Export

In [ ]:
if not use_cluster:
    refine.to_h5()
ds.save()